## Lists and Dictionaries
### Item 13: Prefer Catch-All Unpacking Over Slicing
Basic unpacking has the limitation that you need to know the length of the sequences you're unpacking in advance. 

In [1]:
car_ages = [0, 9, 4, 8, 7, 20, 19, 1, 6, 15]
car_ages_descending = sorted(car_ages, reverse=True)
oldest, second_oldest = car_ages_descending[0], car_ages_descending[1]
others = car_ages_descending[2:]
print(oldest, second_oldest, others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]


This is visuallly noisy as seen above and more prone to off-by-one errors. For example, you might change boundaries on one line but forget to update the others. Using catch all through the *started expression* is a better solution.

In [2]:
oldest, second_oldest, *others = car_ages_descending
print(oldest, second_oldest, others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]


Stared expression can appear in any position, so it is useful when you need to extract one slice:

In [3]:
oldest, *others, youngest = car_ages_descending

*others, second_youngest, youngest = car_ages_descending
print(youngest, second_youngest, others)

0 1 [20, 19, 15, 9, 8, 7, 6, 4]


### Item 14: Sort by Complex Criteria Using the `key` Parameter
When there are multiple sorting criteria, you can use tuples which are comparible by default and have a natural ordering. Natural ordering means that they implement all of the special such as `__lt__`, that are required by the sort method.

In [5]:
class Tool:
    def __init__(self, name, weight):
       self.name = name
       self.weight = weight

    def __repr__(self):
        return f"Tool({self.name!r}, {self.weight!r})"
    
power_tools = [
    Tool("drill", 4),
    Tool("circular saw", 5),
    Tool("jackhammer", 40), 
    Tool("sander", 4)
]

# define a key function that returns a tuple containing the two attributes I want to sort on in order of priority
power_tools.sort(key=lambda x : (x.weight, x.name))
print(power_tools)

[Tool('drill', 4), Tool('sander', 4), Tool('circular saw', 5), Tool('jackhammer', 40)]


A limitation of using the key function to return a tuple is that the direction of sorting is the same for all criteria, i.e. all ascending or descending. However, for numerical data types you can use the unary minus operator in the key function. This will reverse the order of the item it is used on but keep the direction of the rest intact. 

In [6]:
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


The unary operator doesn't work for all types though. Instead, you can apply multiple sorts on the same list and since Python provides a *stable* sorting algorithm the returned order from previous steps is preserved. You just need to make sure that you execute the sorts in the reverse sequence you want the final list to contain. For example, if you want the sort order to be by weight descending and then by name ascending, you have to sort by name first and then weight.

In [8]:
power_tools.sort(key=lambda x: x.name)
power_tools.sort(key=lambda x: x.weight, reverse=True)

print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


### Item 15: Be Cautious Whn Relying on `dict` Insertion Ordering
There are three ways to be careful about dictionary-like classes: 
1. Write code that doesn't rely on insertion ordering,
2. Explicitly check for the dict type at runtime
3. The best way is to require dict values using type annotations and static analysis. 